In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
# calling volumes
subcortical_vol_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\aseg_20231117.csv"
subcortical_vol = pd.read_csv(subcortical_vol_file_path)
subcortical_vol = pd.DataFrame(subcortical_vol)

# Left Hemisphere
lh_aparc_area_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_area_20231117.csv"
lh_aparc_area = pd.read_csv(lh_aparc_area_file_path)
lh_aparc_area = pd.DataFrame(lh_aparc_area)

lh_aparc_meancurv_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_meancurv_20231117.csv"
lh_aparc_meancurv = pd.read_csv(lh_aparc_meancurv_file_path)
lh_aparc_meancurv = pd.DataFrame(lh_aparc_meancurv)

lh_aparc_thickness_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_thickness_20231117.csv"
lh_aparc_thickness = pd.read_csv(lh_aparc_thickness_file_path)
lh_aparc_thickness = pd.DataFrame(lh_aparc_thickness)

lh_aparc_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_volume_20231117.csv"
lh_aparc_volume = pd.read_csv(lh_aparc_volume_file_path)
lh_aparc_volume = pd.DataFrame(lh_aparc_volume)


# Right Hemisphere
rh_aparc_area_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_area_20231117.csv"
rh_aparc_area = pd.read_csv(rh_aparc_area_file_path)
rh_aparc_area = pd.DataFrame(rh_aparc_area)

rh_aparc_meancurv_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_meancurv_20231117.csv"
rh_aparc_meancurv = pd.read_csv(rh_aparc_meancurv_file_path)
rh_aparc_meancurv = pd.DataFrame(rh_aparc_meancurv)

rh_aparc_thickness_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_thickness_20231117.csv"
rh_aparc_thickness = pd.read_csv(rh_aparc_thickness_file_path)
rh_aparc_thickness = pd.DataFrame(rh_aparc_thickness)

rh_aparc_volume_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_volume_20231117.csv"
rh_aparc_volume = pd.read_csv(rh_aparc_volume_file_path)
rh_aparc_volume = pd.DataFrame(rh_aparc_volume)



In [3]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'volumetrics_to_briefa'  
save_path = os.path.join(main_path, folder_name)
# os.makedirs(save_path, exist_ok=True)



# Preparing Subcortical Volume

In [4]:
# Check for null values in each column
subcortical_vol_nulls = subcortical_vol.isnull().sum()

subcortical_vol_nulls_df = pd.DataFrame({
    'Column': subcortical_vol_nulls.index,
    'Number of Nulls': subcortical_vol_nulls.values
})

subcortical_vol_nulls_df_transposed = subcortical_vol_nulls_df.T

print("Null values in subcortical_vol:")
print(subcortical_vol_nulls_df)
subcortical_vol_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\subcortical_vol_nulls.csv", index=False)



Null values in subcortical_vol:
                           Column  Number of Nulls
0                      subject_id                0
1                           visit                0
2                     checkin_bin                0
3                     exposurebin                0
4                      age_decade                0
..                            ...              ...
75                MaskVol_to_eTIV               11
76                 lhSurfaceHoles               11
77                 rhSurfaceHoles               11
78                   SurfaceHoles               11
79  EstimatedTotalIntraCranialVol               11

[80 rows x 2 columns]


In [5]:
subcortical_vol = subcortical_vol.drop(columns=subcortical_vol.loc[:,'subject_id':'visit'].columns)
subcortical_vol = subcortical_vol.drop(columns=subcortical_vol.loc[:,'exposurebin':'aseg_freesurfer_version'].columns)

subcortical_vol = subcortical_vol.drop(columns=['_5th_Ventricle',
                              'Left_WM_hypointensities',
                              'Right_WM_hypointensities', 
                              'non_WM_hypointensities',
                              'Left_non_WM_hypointensities',
                              'Right_non_WM_hypointensities'])

subcortical_vol.head()

subcortical_vol.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 59 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   checkin_bin                    236 non-null    int64  
 1   Left_Lateral_Ventricle         225 non-null    float64
 2   Left_Inf_Lat_Vent              225 non-null    float64
 3   Left_Cerebellum_White_Matter   225 non-null    float64
 4   Left_Cerebellum_Cortex         225 non-null    float64
 5   Left_Thalamus                  225 non-null    float64
 6   Left_Caudate                   225 non-null    float64
 7   Left_Putamen                   225 non-null    float64
 8   Left_Pallidum                  225 non-null    float64
 9   _3rd_Ventricle                 225 non-null    float64
 10  _4th_Ventricle                 225 non-null    float64
 11  Brain_Stem                     225 non-null    float64
 12  Left_Hippocampus               225 non-null    flo

In [6]:
# Calculate the mean for each column, ignoring NaNs
for column in subcortical_vol.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = subcortical_vol.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            subcortical_vol.loc[(subcortical_vol['checkin_bin'] == label) & (subcortical_vol[column].isnull()), column] = averages[label]

# print(bloodbiomk1.head())

In [7]:
subcortical_vol = subcortical_vol.drop(columns='checkin_bin')


# Preparing LH Area

In [8]:
# Check for null values in each column
lh_aparc_area_nulls = lh_aparc_area.isnull().sum()

lh_aparc_area_nulls_df = pd.DataFrame({
    'Column': lh_aparc_area_nulls.index,
    'Number of Nulls': lh_aparc_area_nulls.values
})

lh_aparc_area_nulls_df_transposed = lh_aparc_area_nulls_df.T

print("Null values in lh_aparc_area:")
print(lh_aparc_area_nulls_df)
lh_aparc_area_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_area_nulls.csv", index=False)



Null values in lh_aparc_area:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas               1

In [9]:
lh_aparc_area = lh_aparc_area.drop(columns=lh_aparc_area.loc[:,'subject_id':'visit'].columns)
lh_aparc_area = lh_aparc_area.drop(columns=lh_aparc_area.loc[:,'exposurebin':'Atlas'].columns)

lh_aparc_area.head()

,checkin_bin,lh_bankssts_area,lh_caudalanteriorcingulate_area,lh_caudalmiddlefrontal_area,lh_cuneus_area,lh_entorhinal_area,lh_fusiform_area,lh_inferiorparietal_area,lh_inferiortemporal_area,lh_isthmuscingulate_area,...,lh_rostralmiddlefrontal_area,lh_superiorfrontal_area,lh_superiorparietal_area,lh_superiortemporal_area,lh_supramarginal_area,lh_frontalpole_area,lh_temporalpole_area,lh_transversetemporal_area,lh_insula_area,lh_WhiteSurfArea_area
0,2,1323.0,639.0,2539.0,1848.0,484.0,2887.0,5073.0,3556.0,1157.0,...,6491.0,7849.0,6321.0,4567.0,4577.0,306.0,450.0,559.0,2816.0,96259.6
1,2,1529.0,673.0,3117.0,1740.0,456.0,4163.0,7443.0,4976.0,1114.0,...,7456.0,9472.0,7116.0,5342.0,5733.0,357.0,418.0,542.0,3079.0,114183.0
2,2,868.0,984.0,2142.0,1219.0,399.0,3379.0,4381.0,3335.0,1230.0,...,5240.0,7624.0,4746.0,4317.0,4951.0,249.0,478.0,458.0,2767.0,88801.2
3,1,976.0,749.0,2772.0,1775.0,451.0,3105.0,5084.0,3402.0,1328.0,...,6008.0,8329.0,5315.0,4446.0,5156.0,284.0,527.0,500.0,2558.0,94768.7
4,3,928.0,968.0,1867.0,1519.0,476.0,2924.0,3945.0,2935.0,991.0,...,5624.0,7014.0,5745.0,4135.0,4131.0,261.0,597.0,440.0,2393.0,85044.7


In [10]:
lh_aparc_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 36 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   checkin_bin                       236 non-null    int64  
 1   lh_bankssts_area                  225 non-null    float64
 2   lh_caudalanteriorcingulate_area   225 non-null    float64
 3   lh_caudalmiddlefrontal_area       225 non-null    float64
 4   lh_cuneus_area                    225 non-null    float64
 5   lh_entorhinal_area                225 non-null    float64
 6   lh_fusiform_area                  225 non-null    float64
 7   lh_inferiorparietal_area          225 non-null    float64
 8   lh_inferiortemporal_area          225 non-null    float64
 9   lh_isthmuscingulate_area          225 non-null    float64
 10  lh_lateraloccipital_area          225 non-null    float64
 11  lh_lateralorbitofrontal_area      225 non-null    float64
 12  lh_lingu

In [11]:
# Calculate the mean for each column, ignoring NaNs
for column in lh_aparc_area.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = lh_aparc_area.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            lh_aparc_area.loc[(lh_aparc_area['checkin_bin'] == label) & (lh_aparc_area[column].isnull()), column] = averages[label]



In [12]:
lh_aparc_area = lh_aparc_area.drop(columns='checkin_bin')


# Preparing LH Volume

In [13]:
# Check for null values in each column
lh_aparc_volume_nulls = lh_aparc_volume.isnull().sum()

lh_aparc_volume_nulls_df = pd.DataFrame({
    'Column': lh_aparc_volume_nulls.index,
    'Number of Nulls': lh_aparc_volume_nulls.values
})

lh_aparc_volume_nulls_df_transposed = lh_aparc_volume_nulls_df.T

print("Null values in lh_aparc_volume:")
print(lh_aparc_volume_nulls_df)
lh_aparc_volume_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_volume_nulls.csv", index=False)



Null values in lh_aparc_volume:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas              

In [14]:
lh_aparc_volume = lh_aparc_volume.drop(columns=lh_aparc_volume.loc[:,'subject_id':'visit'].columns)
lh_aparc_volume = lh_aparc_volume.drop(columns=lh_aparc_volume.loc[:,'exposurebin':'Atlas'].columns)

lh_aparc_volume.head()

,checkin_bin,lh_bankssts_volume,lh_caudalanteriorcingulate_volum,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,...,lh_rostralanteriorcingulate_volu,lh_rostralmiddlefrontal_volume,lh_superiorfrontal_volume,lh_superiorparietal_volume,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume
0,2,3044.0,1662.0,6668.0,3864.0,1649.0,7907.0,12750.0,9717.0,2643.0,...,2928.0,16256.0,22036.0,15546.0,13498.0,12041.0,955.0,1601.0,1487.0,8171.0
1,2,3744.0,1675.0,9308.0,3003.0,1432.0,11179.0,19563.0,14240.0,2804.0,...,3649.0,19280.0,28474.0,17270.0,17014.0,15437.0,947.0,1489.0,1317.0,8307.0
2,2,1877.0,2475.0,5777.0,2496.0,1527.0,9419.0,10517.0,9158.0,2746.0,...,2788.0,12913.0,22254.0,11101.0,12622.0,12821.0,851.0,1881.0,1241.0,7678.0
3,1,1979.0,1689.0,6729.0,3676.0,1535.0,8406.0,11853.0,9266.0,2553.0,...,2237.0,14026.0,22113.0,11802.0,11350.0,11963.0,856.0,2156.0,1189.0,6983.0
4,3,2085.0,2423.0,5311.0,3195.0,1741.0,8674.0,10153.0,8315.0,2294.0,...,2501.0,14217.0,19906.0,14458.0,12739.0,10685.0,883.0,2902.0,1148.0,7223.0


In [15]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in lh_aparc_volume.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = lh_aparc_volume.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            lh_aparc_volume.loc[(lh_aparc_volume['checkin_bin'] == label) & (lh_aparc_volume[column].isnull()), column] = averages[label]

# print(lh_aparc_volume.head())

lh_aparc_volume.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\lh_aparc_volume_nullreplaced.csv")


In [16]:
lh_aparc_volume = lh_aparc_volume.drop(columns='checkin_bin')


# Preparing LH Mean Curve

In [17]:
# Check for null values in each column
lh_aparc_meancurv_nulls = lh_aparc_meancurv.isnull().sum()

lh_aparc_meancurv_nulls_df = pd.DataFrame({
    'Column': lh_aparc_meancurv_nulls.index,
    'Number of Nulls': lh_aparc_meancurv_nulls.values
})

lh_aparc_meancurv_nulls_df_transposed = lh_aparc_meancurv_nulls_df.T

print("Null values in lh_aparc_meancurv:")
print(lh_aparc_meancurv_nulls_df)
lh_aparc_meancurv_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_meancurv_nulls.csv", index=False)


Null values in lh_aparc_meancurv:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas            

In [18]:
lh_aparc_meancurv = lh_aparc_meancurv.drop(columns=lh_aparc_meancurv.loc[:,'subject_id':'visit'].columns)
lh_aparc_meancurv = lh_aparc_meancurv.drop(columns=lh_aparc_meancurv.loc[:,'exposurebin':'Atlas'].columns)

lh_aparc_volume.head()

,lh_bankssts_volume,lh_caudalanteriorcingulate_volum,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,lh_lateraloccipital_volume,...,lh_rostralanteriorcingulate_volu,lh_rostralmiddlefrontal_volume,lh_superiorfrontal_volume,lh_superiorparietal_volume,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume
0,3044.0,1662.0,6668.0,3864.0,1649.0,7907.0,12750.0,9717.0,2643.0,14181.0,...,2928.0,16256.0,22036.0,15546.0,13498.0,12041.0,955.0,1601.0,1487.0,8171.0
1,3744.0,1675.0,9308.0,3003.0,1432.0,11179.0,19563.0,14240.0,2804.0,13802.0,...,3649.0,19280.0,28474.0,17270.0,17014.0,15437.0,947.0,1489.0,1317.0,8307.0
2,1877.0,2475.0,5777.0,2496.0,1527.0,9419.0,10517.0,9158.0,2746.0,13697.0,...,2788.0,12913.0,22254.0,11101.0,12622.0,12821.0,851.0,1881.0,1241.0,7678.0
3,1979.0,1689.0,6729.0,3676.0,1535.0,8406.0,11853.0,9266.0,2553.0,11874.0,...,2237.0,14026.0,22113.0,11802.0,11350.0,11963.0,856.0,2156.0,1189.0,6983.0
4,2085.0,2423.0,5311.0,3195.0,1741.0,8674.0,10153.0,8315.0,2294.0,12438.0,...,2501.0,14217.0,19906.0,14458.0,12739.0,10685.0,883.0,2902.0,1148.0,7223.0


In [19]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in lh_aparc_meancurv.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = lh_aparc_meancurv.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            lh_aparc_meancurv.loc[(lh_aparc_meancurv['checkin_bin'] == label) & (lh_aparc_meancurv[column].isnull()), column] = averages[label]

print(lh_aparc_meancurv.head())



   checkin_bin  lh_bankssts_meancurv  lh_caudalanteriorcingulate_meanc  \
0            2                 0.105                             0.107   
1            2                 0.100                             0.096   
2            2                 0.109                             0.146   
3            1                 0.113                             0.141   
4            3                 0.105                             0.134   

   lh_caudalmiddlefrontal_meancurv  lh_cuneus_meancurv  \
0                            0.102               0.126   
1                            0.103               0.147   
2                            0.110               0.126   
3                            0.117               0.138   
4                            0.103               0.136   

   lh_entorhinal_meancurv  lh_fusiform_meancurv  lh_inferiorparietal_meancurv  \
0                   0.134                 0.120                         0.107   
1                   0.130                 0.

In [20]:
lh_aparc_meancurv = lh_aparc_meancurv.drop(columns='checkin_bin')


# Preparing LH Thickness

In [21]:
# Check for null values in each column
lh_aparc_thickness_nulls = lh_aparc_thickness.isnull().sum()

lh_aparc_thickness_nulls_df = pd.DataFrame({
    'Column': lh_aparc_thickness_nulls.index,
    'Number of Nulls': lh_aparc_thickness_nulls.values
})

lh_aparc_thickness_nulls_df_transposed = lh_aparc_thickness_nulls_df.T

print("Null values in lh_aparc_thickness:")
print(lh_aparc_thickness_nulls_df)
lh_aparc_thickness_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\lh_aparc_thickness_nulls.csv", index=False)



Null values in lh_aparc_thickness:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas           

In [22]:
lh_aparc_thickness = lh_aparc_thickness.drop(columns=lh_aparc_thickness.loc[:,'subject_id':'visit'].columns)
lh_aparc_thickness = lh_aparc_thickness.drop(columns=lh_aparc_thickness.loc[:,'exposurebin':'Atlas'].columns)

lh_aparc_thickness.head()

,checkin_bin,lh_bankssts_thickness,lh_caudalanteriorcingulate_thick,lh_caudalmiddlefrontal_thickness,lh_cuneus_thickness,lh_entorhinal_thickness,lh_fusiform_thickness,lh_inferiorparietal_thickness,lh_inferiortemporal_thickness,lh_isthmuscingulate_thickness,...,lh_rostralmiddlefrontal_thicknes,lh_superiorfrontal_thickness,lh_superiorparietal_thickness,lh_superiortemporal_thickness,lh_supramarginal_thickness,lh_frontalpole_thickness,lh_temporalpole_thickness,lh_transversetemporal_thickness,lh_insula_thickness,lh_MeanThickness_thickness
0,2,2.450,2.461,2.575,2.009,2.756,2.678,2.466,2.561,2.180,...,2.427,2.685,2.376,2.807,2.615,2.543,2.939,2.368,3.036,2.51371
1,2,2.672,2.386,2.774,1.821,2.636,2.642,2.546,2.667,2.334,...,2.475,2.808,2.361,2.951,2.624,2.357,2.923,2.441,2.857,2.54090
2,2,2.225,2.342,2.605,1.966,3.001,2.619,2.351,2.553,2.237,...,2.434,2.756,2.290,2.742,2.496,2.699,3.108,2.558,2.907,2.50218
3,1,2.224,2.069,2.378,1.919,2.867,2.463,2.232,2.522,1.891,...,2.270,2.516,2.163,2.401,2.306,2.425,3.081,2.117,2.871,2.31669
4,3,2.460,2.477,2.716,2.023,2.814,2.727,2.465,2.594,2.213,...,2.383,2.649,2.414,2.820,2.541,2.689,3.402,2.571,3.117,2.51057


In [23]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in lh_aparc_thickness.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = lh_aparc_thickness.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            lh_aparc_thickness.loc[(lh_aparc_thickness['checkin_bin'] == label) & (lh_aparc_thickness[column].isnull()), column] = averages[label]

print(lh_aparc_thickness.head())


   checkin_bin  lh_bankssts_thickness  lh_caudalanteriorcingulate_thick  \
0            2                  2.450                             2.461   
1            2                  2.672                             2.386   
2            2                  2.225                             2.342   
3            1                  2.224                             2.069   
4            3                  2.460                             2.477   

   lh_caudalmiddlefrontal_thickness  lh_cuneus_thickness  \
0                             2.575                2.009   
1                             2.774                1.821   
2                             2.605                1.966   
3                             2.378                1.919   
4                             2.716                2.023   

   lh_entorhinal_thickness  lh_fusiform_thickness  \
0                    2.756                  2.678   
1                    2.636                  2.642   
2                    3.001   

In [24]:
lh_aparc_thickness = lh_aparc_thickness.drop(columns='checkin_bin')


# Preparing RH Area

In [25]:
# Check for null values in each column
rh_aparc_area_nulls = rh_aparc_area.isnull().sum()

rh_aparc_area_nulls_df = pd.DataFrame({
    'Column': rh_aparc_area_nulls.index,
    'Number of Nulls': rh_aparc_area_nulls.values
})

rh_aparc_area_nulls_df_transposed = rh_aparc_area_nulls_df.T

print("Null values in rh_aparc_area:")
print(rh_aparc_area_nulls_df)
rh_aparc_area_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_area_nulls.csv", index=False)



Null values in rh_aparc_area:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas               1

In [26]:
rh_aparc_area = rh_aparc_area.drop(columns=rh_aparc_area.loc[:,'subject_id':'visit'].columns)
rh_aparc_area = rh_aparc_area.drop(columns=rh_aparc_area.loc[:,'exposurebin':'Atlas'].columns)

rh_aparc_area.head()
rh_aparc_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 36 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   checkin_bin                       236 non-null    int64  
 1   rh_bankssts_area                  225 non-null    float64
 2   rh_caudalanteriorcingulate_area   225 non-null    float64
 3   rh_caudalmiddlefrontal_area       225 non-null    float64
 4   rh_cuneus_area                    225 non-null    float64
 5   rh_entorhinal_area                225 non-null    float64
 6   rh_fusiform_area                  225 non-null    float64
 7   rh_inferiorparietal_area          225 non-null    float64
 8   rh_inferiortemporal_area          225 non-null    float64
 9   rh_isthmuscingulate_area          225 non-null    float64
 10  rh_lateraloccipital_area          225 non-null    float64
 11  rh_lateralorbitofrontal_area      225 non-null    float64
 12  rh_lingu

In [27]:
# Calculate the mean for each column, ignoring NaNs
for column in rh_aparc_area.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = rh_aparc_area.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            rh_aparc_area.loc[(rh_aparc_area['checkin_bin'] == label) & (rh_aparc_area[column].isnull()), column] = averages[label]



In [28]:
rh_aparc_area = rh_aparc_area.drop(columns='checkin_bin')


# Preparing RH Volume

In [29]:
# Check for null values in each column
rh_aparc_volume_nulls = rh_aparc_volume.isnull().sum()

rh_aparc_volume_nulls_df = pd.DataFrame({
    'Column': rh_aparc_volume_nulls.index,
    'Number of Nulls': rh_aparc_volume_nulls.values
})

rh_aparc_volume_nulls_df_transposed = rh_aparc_volume_nulls_df.T

print("Null values in rh_aparc_volume:")
print(rh_aparc_volume_nulls_df)
rh_aparc_volume_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_volume_nulls.csv", index=False)



Null values in rh_aparc_volume:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas              

In [30]:
rh_aparc_volume = rh_aparc_volume.drop(columns=rh_aparc_volume.loc[:,'subject_id':'visit'].columns)
rh_aparc_volume = rh_aparc_volume.drop(columns=rh_aparc_volume.loc[:,'exposurebin':'Atlas'].columns)

rh_aparc_volume.head()

,checkin_bin,rh_bankssts_volume,rh_caudalanteriorcingulate_volum,rh_caudalmiddlefrontal_volume,rh_cuneus_volume,rh_entorhinal_volume,rh_fusiform_volume,rh_inferiorparietal_volume,rh_inferiortemporal_volume,rh_isthmuscingulate_volume,...,rh_rostralanteriorcingulate_volu,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,2,2424.0,2922.0,5320.0,3853.0,1507.0,9539.0,13387.0,10109.0,2726.0,...,1672.0,15816.0,20720.0,15523.0,14333.0,10071.0,1184.0,1685.0,1095.0,8970.0
1,2,3831.0,3017.0,7612.0,3349.0,1711.0,10800.0,23742.0,11344.0,3040.0,...,2593.0,18136.0,26808.0,17638.0,16253.0,14687.0,1316.0,1339.0,1036.0,8656.0
2,2,2148.0,1981.0,5445.0,3638.0,1562.0,11081.0,14908.0,9899.0,2013.0,...,1713.0,14229.0,21140.0,14419.0,11672.0,9538.0,1102.0,2295.0,877.0,6745.0
3,1,2310.0,1647.0,4656.0,3471.0,1326.0,7496.0,13607.0,8159.0,2354.0,...,1702.0,13092.0,20406.0,14824.0,10843.0,12316.0,958.0,1746.0,1107.0,5813.0
4,3,1790.0,2355.0,4589.0,3667.0,1469.0,8565.0,13656.0,10969.0,2215.0,...,2483.0,15323.0,17971.0,13579.0,12156.0,8855.0,1135.0,2205.0,922.0,7298.0


In [31]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in rh_aparc_volume.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = rh_aparc_volume.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            rh_aparc_volume.loc[(rh_aparc_volume['checkin_bin'] == label) & (rh_aparc_volume[column].isnull()), column] = averages[label]

# print(rh_aparc_volume.head())

rh_aparc_volume.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\rh_aparc_volume_nullreplaced.csv")


In [32]:
rh_aparc_volume = rh_aparc_volume.drop(columns='checkin_bin')


# Preparing RH Mean Curve

In [33]:
# Check for null values in each column
rh_aparc_meancurv_nulls = rh_aparc_meancurv.isnull().sum()

rh_aparc_meancurv_nulls_df = pd.DataFrame({
    'Column': rh_aparc_meancurv_nulls.index,
    'Number of Nulls': rh_aparc_meancurv_nulls.values
})

rh_aparc_meancurv_nulls_df_transposed = rh_aparc_meancurv_nulls_df.T

print("Null values in rh_aparc_meancurv:")
print(rh_aparc_meancurv_nulls_df)
rh_aparc_meancurv_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_meancurv_nulls.csv", index=False)


Null values in rh_aparc_meancurv:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas            

In [34]:
rh_aparc_meancurv = rh_aparc_meancurv.drop(columns=rh_aparc_meancurv.loc[:,'subject_id':'visit'].columns)
rh_aparc_meancurv = rh_aparc_meancurv.drop(columns=rh_aparc_meancurv.loc[:,'exposurebin':'Atlas'].columns)

rh_aparc_meancurv.head()

,checkin_bin,rh_bankssts_meancurv,rh_caudalanteriorcingulate_meanc,rh_caudalmiddlefrontal_meancurv,rh_cuneus_meancurv,rh_entorhinal_meancurv,rh_fusiform_meancurv,rh_inferiorparietal_meancurv,rh_inferiortemporal_meancurv,rh_isthmuscingulate_meancurv,...,rh_rostralanteriorcingulate_mean,rh_rostralmiddlefrontal_meancurv,rh_superiorfrontal_meancurv,rh_superiorparietal_meancurv,rh_superiortemporal_meancurv,rh_supramarginal_meancurv,rh_frontalpole_meancurv,rh_temporalpole_meancurv,rh_transversetemporal_meancurv,rh_insula_meancurv
0,2,0.103,0.113,0.113,0.133,0.128,0.118,0.113,0.119,0.127,...,0.123,0.122,0.118,0.114,0.106,0.117,0.155,0.136,0.110,0.126
1,2,0.096,0.127,0.109,0.135,0.102,0.117,0.109,0.131,0.138,...,0.134,0.125,0.116,0.113,0.106,0.116,0.137,0.146,0.123,0.130
2,2,0.103,0.110,0.114,0.139,0.119,0.124,0.119,0.131,0.127,...,0.136,0.136,0.125,0.111,0.109,0.131,0.160,0.133,0.122,0.125
3,1,0.108,0.112,0.120,0.127,0.123,0.125,0.110,0.145,0.127,...,0.130,0.144,0.119,0.105,0.102,0.128,0.169,0.139,0.095,0.122
4,3,0.101,0.130,0.106,0.129,0.146,0.119,0.119,0.108,0.121,...,0.150,0.134,0.111,0.118,0.101,0.110,0.159,0.145,0.124,0.101


In [35]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in rh_aparc_meancurv.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = rh_aparc_meancurv.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            rh_aparc_meancurv.loc[(rh_aparc_meancurv['checkin_bin'] == label) & (rh_aparc_meancurv[column].isnull()), column] = averages[label]

print(rh_aparc_meancurv.head())



   checkin_bin  rh_bankssts_meancurv  rh_caudalanteriorcingulate_meanc  \
0            2                 0.103                             0.113   
1            2                 0.096                             0.127   
2            2                 0.103                             0.110   
3            1                 0.108                             0.112   
4            3                 0.101                             0.130   

   rh_caudalmiddlefrontal_meancurv  rh_cuneus_meancurv  \
0                            0.113               0.133   
1                            0.109               0.135   
2                            0.114               0.139   
3                            0.120               0.127   
4                            0.106               0.129   

   rh_entorhinal_meancurv  rh_fusiform_meancurv  rh_inferiorparietal_meancurv  \
0                   0.128                 0.118                         0.113   
1                   0.102                 0.

In [36]:
rh_aparc_meancurv = rh_aparc_meancurv.drop(columns='checkin_bin')


# Preparing RH Thickness

In [37]:
# Check for null values in each column
rh_aparc_thickness_nulls = rh_aparc_thickness.isnull().sum()

rh_aparc_thickness_nulls_df = pd.DataFrame({
    'Column': rh_aparc_thickness_nulls.index,
    'Number of Nulls': rh_aparc_thickness_nulls.values
})

rh_aparc_thickness_nulls_df_transposed = rh_aparc_thickness_nulls_df.T

print("Null values in rh_aparc_thickness:")
print(rh_aparc_thickness_nulls_df)
rh_aparc_thickness_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Volumes\rh_aparc_thickness_nulls.csv", index=False)



Null values in rh_aparc_thickness:
                              Column  Number of Nulls
0                         subject_id                0
1                              visit                0
2                        checkin_bin                0
3                        exposurebin                0
4                         age_decade                0
5                   racecat_combined                0
6                           eduyears                0
7                         totyr_foot               56
8                        chiiseas_pf               56
9                         chiiyrs_pf               56
10                       chiiseas_pl               56
11                        chiiyrs_pl               56
12                       chiiseas_pg               56
13                        chiiyrs_pg               56
14                   timepoint_aparc               11
15                FreeSurfer_Version               11
16                             Atlas           

In [38]:
rh_aparc_thickness = rh_aparc_thickness.drop(columns=rh_aparc_thickness.loc[:,'subject_id':'visit'].columns)
rh_aparc_thickness = rh_aparc_thickness.drop(columns=rh_aparc_thickness.loc[:,'exposurebin':'Atlas'].columns)

rh_aparc_thickness.head()

,checkin_bin,rh_bankssts_thickness,rh_caudalanteriorcingulate_thick,rh_caudalmiddlefrontal_thickness,rh_cuneus_thickness,rh_entorhinal_thickness,rh_fusiform_thickness,rh_inferiorparietal_thickness,rh_inferiortemporal_thickness,rh_isthmuscingulate_thickness,...,rh_rostralmiddlefrontal_thicknes,rh_superiorfrontal_thickness,rh_superiorparietal_thickness,rh_superiortemporal_thickness,rh_supramarginal_thickness,rh_frontalpole_thickness,rh_temporalpole_thickness,rh_transversetemporal_thickness,rh_insula_thickness,rh_MeanThickness_thickness
0,2,2.613,2.358,2.591,2.040,2.768,2.753,2.437,2.559,2.156,...,2.340,2.604,2.373,2.789,2.578,2.647,2.856,2.805,3.012,2.48321
1,2,2.837,2.243,2.610,1.738,2.905,2.731,2.549,2.677,2.315,...,2.391,2.695,2.334,2.981,2.759,2.614,2.617,2.285,2.800,2.51874
2,2,2.601,2.098,2.600,2.128,3.153,2.634,2.402,2.607,2.150,...,2.283,2.694,2.330,2.969,2.546,2.626,3.579,2.257,3.031,2.48204
3,1,2.481,2.015,2.348,1.894,2.716,2.371,2.361,2.437,2.002,...,2.267,2.465,2.128,2.527,2.474,2.311,2.946,2.302,2.808,2.34217
4,3,2.483,2.336,2.462,2.061,2.624,2.837,2.431,2.745,2.177,...,2.355,2.639,2.351,2.875,2.378,2.656,2.985,2.806,2.928,2.49441


In [39]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in rh_aparc_thickness.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = rh_aparc_thickness.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            rh_aparc_thickness.loc[(rh_aparc_thickness['checkin_bin'] == label) & (rh_aparc_thickness[column].isnull()), column] = averages[label]

print(rh_aparc_thickness.head())


   checkin_bin  rh_bankssts_thickness  rh_caudalanteriorcingulate_thick  \
0            2                  2.613                             2.358   
1            2                  2.837                             2.243   
2            2                  2.601                             2.098   
3            1                  2.481                             2.015   
4            3                  2.483                             2.336   

   rh_caudalmiddlefrontal_thickness  rh_cuneus_thickness  \
0                             2.591                2.040   
1                             2.610                1.738   
2                             2.600                2.128   
3                             2.348                1.894   
4                             2.462                2.061   

   rh_entorhinal_thickness  rh_fusiform_thickness  \
0                    2.768                  2.753   
1                    2.905                  2.731   
2                    3.153   

In [40]:
rh_aparc_thickness = rh_aparc_thickness.drop(columns='checkin_bin')


# Concatenating all MRI Metrics

In [41]:
import pandas as pd
MRI_Metrics = pd.concat([subcortical_vol, lh_aparc_area, lh_aparc_meancurv, lh_aparc_thickness, lh_aparc_volume, rh_aparc_area, rh_aparc_meancurv, rh_aparc_thickness, rh_aparc_volume], axis=1)
# print(MRI_Metrics)


In [42]:
MRI_Metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Columns: 334 entries, Left_Lateral_Ventricle to rh_insula_volume
dtypes: float64(334)
memory usage: 615.9 KB


# Now Calling Neuro-related Matrics

In [43]:
# calling Neuropsychiatric Measurements

briefa_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_20231117.csv"
briefa = pd.read_csv(briefa_file_path)
briefa = pd.DataFrame(briefa)


In [44]:
# Check for null values in each column
briefa_nulls = briefa.isnull().sum()

briefa_nulls_df = pd.DataFrame({
    'Column': briefa_nulls.index,
    'Number of Nulls': briefa_nulls.values
})

briefa_nulls_df_transposed = briefa_nulls_df.T

print("Null values in briefa:")
print(briefa_nulls_df)
briefa_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_nulls.csv", index=False)


Null values in briefa:
          Column  Number of Nulls
0     subject_id                0
1          visit                0
2    checkin_bin                0
3    exposurebin                0
4     age_decade                0
..           ...              ...
102      workorg                0
103    upseteasy                0
104    impulsive                0
105       pickup                0
106     complete                0

[107 rows x 2 columns]


In [45]:
briefa.head()

,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,activityorg,getover,onething,moodchange,consequence,workorg,upseteasy,impulsive,pickup,complete
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,2,1,2,1,1,2,2,1,1,2
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,3,3,3,2,3,3,3,3,2,3
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,3,3,3,3,2,3,3,3,1,2
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,1,1,1,1,1,1,2,1,1,1
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,1,2,1,1,1,1,1,1,1,1


In [46]:
briefa = briefa.drop(columns=briefa.loc[:,'subject_id':'chiiyrs_pg'].columns)
briefa = briefa.drop(columns=briefa.loc[:,'negativ':'complete'].columns)
briefa.head()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
0,75,64,114,57,39,46,12,9,11,7,14,17,20,13,11
1,101,85,171,85,70,79,17,15,23,15,19,24,27,15,16
2,90,77,158,79,68,77,15,14,27,12,20,22,22,14,12
3,43,40,81,42,38,46,9,8,15,6,9,8,11,7,8
4,43,39,81,41,38,45,9,11,12,6,10,8,10,7,8


In [47]:
briefa.describe()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
count,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000
mean,63.682203,56.724576,110.169492,56.508475,46.487288,55.203390,12.169492,9.690678,15.758475,8.868644,12.605932,13.758475,15.338983,9.466102,12.512712
std,19.879120,16.771696,33.473344,17.217382,14.657067,16.438698,3.606861,3.378543,5.769042,3.063115,4.101039,4.922909,5.199379,2.946855,4.335387
min,40.000000,36.000000,70.000000,35.000000,30.000000,36.000000,8.000000,6.000000,10.000000,6.000000,8.000000,8.000000,10.000000,6.000000,8.000000
25%,45.000000,41.000000,79.750000,41.000000,34.000000,41.000000,9.000000,6.000000,10.000000,6.000000,9.000000,9.000000,10.000000,6.000000,9.000000
50%,60.000000,53.000000,104.500000,54.000000,43.000000,51.000000,11.000000,9.000000,14.000000,8.000000,12.000000,13.000000,14.000000,9.000000,11.000000
75%,79.250000,68.250000,136.000000,69.000000,57.000000,67.250000,14.000000,12.000000,20.000000,11.000000,16.000000,18.000000,19.250000,12.000000,15.000000
max,116.000000,104.000000,204.000000,108.000000,88.000000,105.000000,23.000000,18.000000,30.000000,18.000000,23.000000,24.000000,29.000000,18.000000,24.000000


In [48]:
MRI_Metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Columns: 334 entries, Left_Lateral_Ventricle to rh_insula_volume
dtypes: float64(334)
memory usage: 615.9 KB


In [49]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_MRI_Metrics'
folder_name = 'MRI_Metrics_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)


In [50]:
categories_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\categories.csv"
categories_df = pd.read_csv(categories_file_path)
new_column = categories_df['checkin_bin']
briefa['category'] = new_column
print(briefa.head())

MRI_Metrics['category'] = new_column
# print(MRI_Metrics.head())


    mi  tmi  gec  tgec  bri  tbri  inhibit  shift  emotcont  selfmon  \
0   75   64  114    57   39    46       12      9        11        7   
1  101   85  171    85   70    79       17     15        23       15   
2   90   77  158    79   68    77       15     14        27       12   
3   43   40   81    42   38    46        9      8        15        6   
4   43   39   81    41   38    45        9     11        12        6   

   initiate  workmem  planorg  taskmon  orgmat  category  
0        14       17       20       13      11         2  
1        19       24       27       15      16         2  
2        20       22       22       14      12         2  
3         9        8       11        7       8         1  
4        10        8       10        7       8         3  



# 95th percentile for both analysis and plotting

In [51]:
for feature in MRI_Metrics.columns[:-1]:  # Assuming 'category' is the last column
    upper_limit = MRI_Metrics[feature].quantile(0.95)  # Cap at 95th percentile
    lower_limit = MRI_Metrics[feature].quantile(0.05)  # Floor at 5th percentile
    MRI_Metrics[feature] = np.where(MRI_Metrics[feature] > upper_limit, upper_limit, MRI_Metrics[feature])
    MRI_Metrics[feature] = np.where(MRI_Metrics[feature] < lower_limit, lower_limit, MRI_Metrics[feature])

    

# Initial Step for Trainings

In [52]:
X = MRI_Metrics.copy()

In [53]:
X

,Left_Lateral_Ventricle,Left_Inf_Lat_Vent,Left_Cerebellum_White_Matter,Left_Cerebellum_Cortex,Left_Thalamus,Left_Caudate,Left_Putamen,Left_Pallidum,_3rd_Ventricle,_4th_Ventricle,...,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume,category
0,18220.0,427.400,22034.9,53338.40,8148.5,3610.7,4973.10,2021.2,2076.400,2084.800,...,15816.00,20720.0,15523.00,13918.25,10071.0,1184.0,1702.0,1095.0,8367.5,2
1,7400.7,563.800,21653.7,57444.90,8574.0,4368.0,5785.50,2338.1,817.900,1769.000,...,18136.00,25051.5,16821.25,13918.25,12368.5,1316.0,1702.0,1036.0,8367.5,2
2,8437.0,282.600,15157.0,48220.10,6582.3,2920.5,4881.80,1851.8,749.475,1216.600,...,14229.00,21140.0,14419.00,11672.00,9538.0,1102.0,2295.0,877.0,6745.0,2
3,21732.3,821.900,17449.0,58175.75,7947.8,3150.6,4049.65,2202.7,1932.000,1800.000,...,13092.00,20406.0,14824.00,10843.00,12316.0,958.0,1746.0,1107.0,5813.0,1
4,19382.7,814.100,19089.0,52633.40,6623.0,3381.0,4439.20,2160.1,1819.800,2387.500,...,15323.00,17971.0,13579.00,12156.00,8855.0,1135.0,2205.0,922.0,7298.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,7723.0,618.900,17605.5,48363.20,7859.3,3450.5,4456.60,1743.3,1073.100,1495.700,...,11918.75,19033.0,10662.00,12454.00,9778.0,1029.0,2248.0,828.0,6993.0,2
232,7180.9,297.400,22986.4,49637.20,7405.4,3179.0,4574.10,1906.8,1086.500,983.075,...,15107.00,19538.0,11602.00,13312.00,11566.0,851.5,2037.0,1102.0,7635.0,3
233,7504.1,384.400,19169.2,48397.10,7215.8,2921.2,5251.70,2275.1,1106.700,1510.900,...,14673.00,22377.0,12209.00,11778.00,11976.0,1220.0,1808.0,1117.0,6016.0,2
234,27979.9,1341.325,18540.5,52071.20,7213.7,3495.9,4733.00,1986.5,2570.950,2450.900,...,14911.00,22535.0,11825.00,10562.00,8917.0,1256.0,2428.0,842.0,6186.0,1


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# got error due to non numeric values, so removing them:

# Assume bloodbiomk1 is your DataFrame loaded with various types of data
X = MRI_Metrics.copy()

# Select only numeric columns from the DataFrame
X_numeric = X.select_dtypes(include=[np.number])
X = X_numeric

# Now X_numeric contains only the numeric columns of the original DataFrame
# Proceed with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)


models = {}
predictions = {}


X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)

In [55]:
## Since I got error about duplicate columns, I am removing them here:
# Check for duplicate column names
duplicate_columns = X.columns[X.columns.duplicated()].tolist()
if duplicate_columns:
    print("Duplicate columns found:", duplicate_columns)

# Drop duplicate columns if desired
X = X.loc[:, ~X.columns.duplicated()]


In [56]:
# Define the columns you want to select for modeling
## columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat'] 
selected_columns = ['selfmon']  # Replace 'column1' and 'column2' with the actual column names

# selected_columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat']  # Replace 'column1' and 'column2' with the actual column names


In [57]:
selected_columns

['selfmon']

In [58]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Initialize an empty list to store results
results = []
feature_importance_rankings = {}
model_name = "Random Forest"
selected_columns_str = "_".join(selected_columns)

# First, calculate feature importances for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Get feature importance and rank features
    feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Save performance metrics with all features as a baseline
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")


# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to feature_importance_rankings.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")



Feature importance rankings saved to feature_importance_rankings.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [59]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Multiple Linear Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for linear regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [60]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression


# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gradient Boosting Machines"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LGBMRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001726 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15234
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 335
[LightGBM] [Info] Start training from score 8.890909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000020 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 1
[LightGBM] [Info] Start training from score 8.890909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000039 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 234
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 8
[LightGBM] [Info] Start training from score 8.890909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 605
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 16
[LightGBM] [Info] Start training from score 8.890909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000876 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14816
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 326
[LightGBM] [Info] Start training from score 8.890909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001151 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15020
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 330
[LightGBM] [Info] Start training from score 8.890909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [61]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "XGBoost"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = xgb.XGBRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [62]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Support Vector Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for SVR
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train SVR model with all features as a baseline
    model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [63]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "ANN"
selected_columns_str = "_".join(selected_columns)

# Standardize features for ANN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train ANN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=X_train_selected.shape[1]),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_selected, y_train, epochs=100, batch_size=10, verbose=0)
    
    y_pred = model.predict(X_test_selected).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Scale selected features for ANN
        X_train_selected_scaled = scaler.fit_transform(X_train_selected)
        X_test_selected_scaled = scaler.transform(X_test_selected)
        
        # Train and evaluate ANN model with selected features
        model = Sequential([
            Dense(128, activation='relu', input_dim=X_train_selected_scaled.shape[1]),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_selected_scaled, y_train, epochs=100, batch_size=10, verbose=0)
        
        y_pred = model.predict(X_test_selected_scaled).flatten()
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


3/3 [==============================] - 0s 999us/step


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


3/3 [==============================] - 0s 3ms/step


3/3 [==============================] - 0s 1ms/step


3/3 [==============================] - 0s 1ms/step
Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [64]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "KNeighbors Regressor"
selected_columns_str = "_".join(selected_columns)

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train KNN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate KNN model with selected features
        model = KNeighborsRegressor(n_neighbors=5)
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [65]:
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Stacking Regressor"
selected_columns_str = "_".join(selected_columns)

# Define the base models and final estimator for stacking
base_models = [
    ('knn', KNeighborsRegressor(n_neighbors=13, leaf_size=23, p=1, weights='uniform')),
    ('svr', SVR(kernel='rbf', C=0.2, epsilon=0.01))
]
final_estimator = Ridge()
stack_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator, cv=5)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train stacking model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    stack_model.fit(X_train_selected, y_train)
    y_pred = stack_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate stacking model with selected features
        stack_model.fit(X_train_selected, y_train)
        y_pred = stack_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [66]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [67]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [68]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Lasso Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Lasso regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Lasso Regression Model with L1 regularization for feature selection
    model = Lasso(alpha=0.1)
    model.fit(X_train, y_train)
    
    # Select features based on non-zero coefficients
    selector = SelectFromModel(model, prefit=True)
    selected_features = X_train.columns[selector.get_support()]
    
    # Feature importance ranking based on absolute Lasso coefficients
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features

    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Lasso(alpha=0.1)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [69]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize lists and dictionaries to store results and feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Ridge Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Ridge regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Ridge Regression Model
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Ridge(alpha=1.0)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [70]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Initialize lists and dictionaries for storing results and feature rankings
results = []
feature_importance_rankings = {}
model_name = "Bayesian Ridge"
selected_columns_str = "_".join(selected_columns)

# Standardize features for consistency
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = BayesianRidge()
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance ranking
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = BayesianRidge()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [71]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gaussian Process Regression"
selected_columns_str = "_".join(selected_columns)

# Define the kernel
kernel = RBF(1.0)

# Standardize features for Gaussian Process Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train the full Gaussian Process model as a baseline
    model = GaussianProcessRegressor(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_MRI_Metrics_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = GaussianProcessRegressor(kernel=kernel, random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_MRI_Metrics_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Lo

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessi

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling f

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
  

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx
